In [1]:
#!pip install webdriver_manager --quiet
#!pip install --upgrade webdriver_manager
#!pip install -U selenium --quiet

In [1]:
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import os

import time
from utilities import *

In [2]:
from datetime import datetime, timedelta

In [3]:
from constants import *

In [4]:
#from constants_negative import *

In [5]:
fb_links  = [hjh_fb,apc_fb,akw_fb,adc_fb,alw_fb,ahc_fb,aab_fb,and_fb,ksa_care_fb,med_tour_fb,and_egy_fb,and_vis_fb,and_car_fb, and_home_fb,dot_care_fb];        fb_pages  = ['FaceBook HJH','FaceBook APC','FaceBook AKW','FaceBook ADC','FaceBook ALW','FaceBook AHC','FaceBook AAB','FaceBook Andalusia_Group','FaceBook KSA_HomeCare','FaceBook Medical_Tour','FaceBook Andalusia_Egypt','FaceBook Andalusia_Visiting','FaceBook Andalusia_careers','FaceBook Homecare','FaceBook Dotcare']

ins_links = [hjh_ins,apc_ins,akw_ins,adc_ins,alw_ins,ahc_ins,and_ins,med_tour_ins,and_egy_ins,and_vis_ins,and_car_ins,and_home_ins,dot_care_ins,dot_care_ksa_ins];  ins_pages = ['Instagram HJH','Instagram APC','Instagram AKW','Instagram ADC','Instagram ALW','Instagram AHC','Instagram Andalusia_Group','Instagram Medical_Tour','Instagram Andalusia_Egypt','Instagram Andalusia_Visiting','Instagram Andalusia_careers','Instagram Homecare','Instagram Dotcare','Instagram KSA_Dotcare']

yt_links  = [akw_yt,adc_yt,and_egy_yt]; yt_pages = ['Youtube AKW','Youtube ADC','Youtube Andalusia_Egypt']

try:
    twt_links = [hjh_twt,apc_twt,akw_twt,adc_twt,alw_twt];          twt_pages = ['Twitter HJH','Twitter APC','Twitter AKW','Twitter ADC','Twitter ALW','Twitter AHC']
except:
    twt_links = []; twt_pages = []

lnk_links = [and_egy_lnk,aab_lnk,and_lnk,and_car_lnk]; lnk_pages = ['LinkedIn Andalusia_Egypt','LinkedIn AAB','LinkedIn Andalusia_Group','LinkedIn Andalusia_careers']

In [13]:
driver = webdriver.Chrome()

In [14]:
#driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

driver.get("https://app.agorapulse.com/")

In [15]:
file_path = 'credentials.txt'
file1 = open(file_path, 'r')
content = file1.read()

content = content.split('\n')
email,password = content[0],content[1]

In [16]:
htmlstring = driver.page_source
soup = BeautifulSoup(htmlstring, "html.parser")

#item = soup.findNext('input',name_="email")

In [17]:
def log_in(driver):
    button = driver.find_element(By.NAME,'email')
    button.send_keys(email)

    button = driver.find_element(By.NAME,'password')
    button.send_keys(password)

    button = driver.find_element(By.NAME,'auth-login-button')
    button.click()

log_in(driver)

In [18]:
print(email)

Analytics-Group@Andalusiagroup.net


In [19]:
num_links = len(fb_links + twt_links + ins_links + yt_links + lnk_links)
## len(fb_links + ins_links + yt_links)

num_links

41

In [20]:
import re

def gm_date(rev_date, needed_date):
    days_vol = {"day": 1, "week": 7, "month": 30, "year": 365}
    text = rev_date.split()
    text[0] = re.sub(r'an?', '1', text[0])
    text[1] = text[1].rstrip('s')
    num_hours = int(text[0]) * int(text[1] == "hour")

    try:
        duration = days_vol[text[1]]
    except KeyError:
        duration = 0
    num_days = int(text[0]) * duration

    return get_day(needed_date, num_days, num_hours, day_only=False)


def get_day(needed_date, past_days=0, past_hours=0, day_only=True, ):
    ret = needed_date - timedelta(days=past_days, hours=past_hours)

    return ret.date() if day_only else ret

In [21]:
all_links = fb_links + twt_links + ins_links + yt_links + lnk_links
all_pages = fb_pages + twt_pages + ins_pages + yt_pages + lnk_pages

In [22]:
day_time_sec = 60 * 60 * 24
day_time_sec

86400

In [23]:
two_three = day_time_sec*2/3
for week in range(1):

    time.sleep(two_three)

    comments = []; days = [];
    names = []; indices = [];
    dates_time = []

    for i in range(num_links):
        link = all_links[i]
        driver.get(link)
        time.sleep(20)

        htmlstring = driver.page_source
        soup = BeautifulSoup(htmlstring, "html.parser")

        long_scroll(driver=driver)

        mydivs = soup.find('div', class_='item-list-cards-container')

        comments.append([]);  days.append([]);
        names.append([]);  dates_time.append([])
        indices.append(all_pages[i])

        if mydivs:
            for div in mydivs:
                try:
                    name = div.findNext("span", attrs={"class": "user-name"}).text.strip()
                    day = div.findNext("span", attrs={"class": "creation-time-ago ng-star-inserted"}).text.strip()
                    comment = div.findNext("span", attrs={"class": "item-card-message ng-star-inserted"}).text.strip()
                    current_date = datetime.now().date()

                    if (len(comments[i]) == 0):
                        names[i].append(name);
                        days[i].append(day);
                        comments[i].append(comment)
                        dates_time[i].append(current_date)
                    elif (comments[i][-1] != comment):
                        names[i].append(name);
                        days[i].append(day);
                        comments[i].append(comment)
                        dates_time[i].append(current_date)
                except:
                    j = i
        else:
            comments[i] = [''];
            days[i] = [''];
            names[i] = [''];
            dates_time[i] = ['']

    indices_column = [];
    names_column = []
    comments_column = [];
    days_column = [];
    dates_column = []

    for i in range(num_links):
        if len(comments[i]) > 0:
            for j in range(len(comments[i])):
                indices_column.append(indices[i])
                comments_column.append(comments[i][j]);  dates_column.append(dates_time[i][j])
                names_column.append(names[i][j]);  days_column.append(days[i][j])

    platform, Business_Units = [], []

    for i in range(len(indices_column)):
        social_media, bu = indices_column[i].split(' ')[0], indices_column[i].split(' ')[1]
        platform.append(social_media);
        Business_Units.append(bu)

    df = pd.DataFrame()

    df['Platform'] = platform;   df['Business Unit'] = Business_Units
    df['message'] = comments_column;    df['Time ago'] = days_column
    df['Comment Author'] = names_column;   df['Current date'] = dates_column

    current_datetime = datetime.now()
    ## modified datetime is approximately 15 minutes this is almost the time taken from the beginning of the scrap to reach
    ##into here
    modified_datetime = current_datetime - timedelta(minutes=15)

    time_ago = df['Time ago'].tolist()

    dates_column = []
    for i in range(len(time_ago)):
        try:
            dates_column.append(gm_date(time_ago[i], modified_datetime))
        except:
            dates_column.append(modified_datetime)

    df['Current date'] = dates_column
    df.drop(columns=['Time ago'], inplace=True)

    datetime_str = current_datetime - timedelta(days=1,minutes=50)
    certain_date = pd.to_datetime(datetime_str)

    df = df[df['Current date'] >= certain_date]
    df = df[df['message'] != '']
    df.drop_duplicates(inplace=True)

    directory_path = 'Extracted SocialMediaComments/' + str(current_date)
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    directory_path += '/Data_Shown.csv'
    df.to_csv(directory_path, encoding='utf-8-sig', index=False)

In [13]:
comments = []; indices = []

for i in range(num_links):
    link = all_links[i]
    driver.get(link)
    time.sleep(2)

    htmlstring = driver.page_source
    soup = BeautifulSoup(htmlstring, "html.parser")
    #wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'ng-input')))
    #mydivs = soup.findAll("span",class_="item-card-message ng-star-inserted")

    #long_scroll(driver=driver)
    time.sleep(1)
    '''
    try:
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME,'ng-input')))
    except:
        time.sleep(5)
    '''
    mydivs = soup.findAll("span",class_="item-card-message ng-star-inserted")
    time.sleep(1)
    #wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME,'ng-input')))
    #mydivs2 = soup.findAll("span",class_="inbox-item-list-card active")

    mydivs = list(mydivs)
    comments.append([])
    indices.append(all_pages[i])

    if len(mydivs) >0:
         for div in mydivs:
             try:
                comments[i].append(str(div).split('">')[-1].split('<')[0].strip())
             except:
                j = i
    else:
        comments[i] = ['']

indices_column = []; comments_column = []
for i in range(num_links):
    if len(comments[i]) > 0:
        for comm in comments[i]:
            indices_column.append(indices[i])
            comments_column.append(comm)

platform, Business_Units = [], []

for i in range(len(indices_column)):
    social_media, bu = indices_column[i].split(' ')[0], indices_column[i].split(' ')[1]
    platform.append(social_media);
    Business_Units.append(bu)

df = pd.DataFrame()
df['Platform'] = platform
df['Business Unit'] = Business_Units
df['message'] = comments_column

df = df[df['message'] != '']
df.to_csv('Extracted SocialMediaComments/All data /v1.csv', encoding='utf-8-sig', index=False)

In [15]:
comments

[['كم سعر الجلسه',
  'قول كم السعر يمكن ما يعجبني',
  'كم السعر',
  'Andalusia Hospital - مستشفى أندلسية يجب أن تكون الردود في العام حتى يستفيد الجميع',
  'مش حايردو عليكم !!! لاتتعبوا نفسكم'],
 ['الموقع', 'كم السعر', 'كم السعر', 'بكم'],
 [''],
 [''],
 [''],
 ['تفاصيل', 'ممكن تفاصيل', 'تفاصيل', 'تم', 'تفاصيل', 'تفاصيل'],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['How much is hydra facial?',
  'كم سعر الجلسه',
  'مانوع التقنيه وكم سعرها',
  'محد تواصل للان'],
 [''],
 [''],
 ['']]

In [16]:
indices

['FaceBook HJH',
 'FaceBook APC',
 'FaceBook AKW',
 'FaceBook ADC',
 'FaceBook ALW',
 'FaceBook AHC',
 'Twitter HJH',
 'Twitter APC',
 'Twitter AKW',
 'Twitter ADC',
 'Twitter ALW',
 'Twitter AHC',
 'Instagram HJH',
 'Instagram APC',
 'Instagram AKW',
 'Instagram ADC',
 'Instagram ALW',
 'Instagram AHC',
 'Youtube AKW']

In [71]:
from datetime import datetime, timedelta

current_date = datetime.now().date()
current_date_column = [current_date]*len(platform)

In [27]:
all_links

['https://app.agorapulse.com/inboxV2/facebook/330347?view=comment&tab=allItems&filter=approvedByManagerId&value=0&item=2394825504019216_830619271908350',
 'https://app.agorapulse.com/inboxV2/facebook/341344?view=comment&tab=allItems&filter=approvedByManagerId&value=0&item=3412196358997462_997347901692276',
 'https://app.agorapulse.com/inboxV2/facebook/330346?view=comment&tab=allItems&filter=approvedByManagerId&value=0',
 'https://app.agorapulse.com/inboxV2/facebook/170260?view=comment&tab=allItems&filter=approvedByManagerId&value=0&item=805982000903409_664860178444810',
 'https://app.agorapulse.com/inboxV2/facebook/447223?tab=allItems&filter=approvedByManagerId',
 'https://app.agorapulse.com/inboxV2/facebook/351704?view=comment&tab=allItems&filter=approvedByManagerId&value=0&item=6451003948293251_821072306336526',
 'https://app.agorapulse.com/inboxV2/twitter/84219?view=mention&tab=allItems&filter=approvedByManagerId&value=0',
 'https://app.agorapulse.com/inboxV2/twitter/84434?view=ment

In [35]:
for i in range(num_links):
    link = all_links[i]
    driver.get(link)

    time.sleep(2)

In [37]:
    mydivs = soup.find('div',class_='item-list-cards-container')
    mydivs

<div _ngcontent-dqt-c459="" class="item-list-cards-container">
<ap-inbox-item-list-card _ngcontent-dqt-c459="" _nghost-dqt-c454="" class="item-entry-container ng-star-inserted" data-test="inbox-listItem-card" id="inboxListItemCardTrack"><div _ngcontent-dqt-c454="" class="inbox-item-list-card" mat-list-item="" selected="false" tabindex="-1">
<div _ngcontent-dqt-c454="" class="item-card-header">
<ap-avatar _ngcontent-dqt-c454="" _nghost-dqt-c135=""><div _ngcontent-dqt-c135="" class="ap-avatar-container medium-sm" style="width: 36px;"><div _ngcontent-dqt-c135="" class="ng-star-inserted"><img _ngcontent-dqt-c135="" border="" class="ap-avatar medium-sm outer-border rounded ng-star-inserted" height="36" loading="auto" src="https://yt3.ggpht.com/ytc/AOPolaS6fA2zh2xkU9L68-lsZ2knSQDiusIjbu4LRRb3rFKA7eo7hppjAnFcpt2f0fIy=s48-c-k-c0x00ffffff-no-rj" width="36"/><!-- --><!-- --></div><!-- --><!-- --><div _ngcontent-dqt-c135="" class="medium-sm social-symbol-wrapper youtube ng-star-inserted"><ap-symb

In [39]:
len(mydivs)

11

In [ ]:
current_datetime = datetime.now()

modified_datetime = current_datetime - timedelta(minutes=10)


time_ago = df['Time ago'].tolist()

dates_column = []

for i in range(len(time_ago)):
        dates_column.append(gm_date(time_ago[i], modified_datetime))

df['Current date'] = dates_column
df.drop(columns=['Time ago'], inplace=True)

df

In [19]:
import re

def gm_date(rev_date, needed_date):
    days_vol = {"day": 1, "week": 7, "month": 30, "year": 365}
    text = rev_date.split()
    text[0] = re.sub(r'an?', '1', text[0])
    text[1] = text[1].rstrip('s')
    num_hours = int(text[0]) * int(text[1] == "hour")

    try:
        duration = days_vol[text[1]]
    except KeyError:
        duration = 0
    num_days = int(text[0]) * duration

    return get_day(needed_date, num_days, num_hours, day_only=False)


def get_day(needed_date, past_days=0, past_hours=0, day_only=True, ):
    ret = needed_date - timedelta(days=past_days, hours=past_hours)

    return ret.date() if day_only else ret




In [20]:
day_time_sec = 60*60*24

In [ ]:
comments = []; days = []; names = []; indices = [];dates_time = []

for looper in range(5):
    time.sleep(day_time_sec)
    for i in range(num_links):
        link = all_links[i]
        driver.get(link)
        time.sleep(20)

        htmlstring = driver.page_source
        soup = BeautifulSoup(htmlstring, "html.parser")

        long_scroll(driver=driver)

        #mydivs = soup.findAll("ap-inbox-item-list-card",class_="item-entry-container item-selected ng-star-inserted")
        mydivs = soup.find('div',class_='item-list-cards-container')

        comments.append([]); days.append([]); names.append([]); dates_time.append([])
        indices.append(all_pages[i])

        if len(mydivs) > 0:
            for div in mydivs:
                try:
                    name = div.findNext("span",attrs={"class":"user-name"}).text.strip()
                    day = div.findNext("span",attrs={"class":"creation-time-ago ng-star-inserted"}).text.strip()
                    comment = div.findNext("span",attrs={"class":"item-card-message ng-star-inserted"}).text.strip()
                    current_date = datetime.now().date()

                    if ( len(comments[i]) == 0):
                        names[i].append(name);  days[i].append(day); comments[i].append(comment)
                        dates_time[i].append(current_date)
                    elif ( comments[i][-1] != comment):
                        names[i].append(name);  days[i].append(day); comments[i].append(comment)
                        dates_time[i].append(current_date)
                except:
                        j = i
        else:
            comments[i] = ['']; days[i] = ['']; names[i] = ['']; dates_time[i] = ['']

    indices_column = []; names_column = []
    comments_column = []; days_column = []; dates_column = []

    for i in range(num_links):
        if len(comments[i]) > 0:
            for j in range(len(comments[i])):
                indices_column.append(indices[i])
                comments_column.append(comments[i][j]) ; dates_column.append(dates_time[i][j])
                names_column.append(names[i][j]);  days_column.append(days[i][j])

    platform, Business_Units = [], []

    for i in range(len(indices_column)):
        social_media, bu = indices_column[i].split(' ')[0], indices_column[i].split(' ')[1]
        platform.append(social_media); Business_Units.append(bu)

    df = pd.DataFrame()

    df['Platform'] = platform
    df['Business Unit'] = Business_Units
    df['message'] = comments_column
    df['Time ago'] = days_column
    df['Comment Author'] = names_column
    df['Current date'] = dates_column

    current_datetime = datetime.now()
    modified_datetime = current_datetime - timedelta(minutes=5)

    time_ago = df['Time ago'].tolist()
    dates_column = []

    for i in range(len(time_ago)):
        dates_column.append(gm_date(time_ago[i], modified_datetime))

    df['Current date'] = dates_column
    df.drop(columns=['Time ago'], inplace=True)


    datetime_str = current_datetime - timedelta(days=1)
    certain_date = pd.to_datetime(datetime_str)

    df = df[df['Current date'] >= certain_date]
    df.drop_duplicates(inplace=True)

    directory_path = 'Extracted SocialMediaComments/' + str(current_date)
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    directory_path += '/Data_Shown.csv'
    df.to_csv(directory_path, encoding='utf-8-sig', index=False)

In [48]:
certain_date = pd.to_datetime('2023-07-17')
filtered_df = df[df['Current date'] >= certain_date]


,Platform,Business Unit,Comment / Mention,Comment Author,Current date
1,FaceBook,APC,الموقع,مالك ومروان,2023-07-17 09:21:02.913435
4,FaceBook,ADC,السلام عليكم كم الحشو,ابو مصطفي ابو مصطفي,2023-07-17 18:21:02.913435
5,FaceBook,ADC,كم سعر السن الواحد,أ.احمد الوا حدي,2023-07-17 18:21:02.913435
6,FaceBook,ALW,عبارة عن ايه الجلسة من فضلك,Ghada Ibrahim,2023-07-17 18:21:02.913435
18,Instagram,ADC,ما العروض المتاحة علي تنظيف الاسنان,ysraaltayeb,2023-07-17 18:21:02.913435
19,Instagram,ADC,Okay,iinada02,2023-07-17 18:21:02.913435
20,Instagram,ADC,ما العروض المتاحة علي ال so check ؟,omalaa___13,2023-07-17 18:21:02.913435
21,Instagram,ADC,كيف احجز لاني من المدية رايحة جدة الاربعاء,anahappy6677,2023-07-17 18:21:02.913435
22,Instagram,ADC,كيف اخسر وزني؟؟,rmharasani,2023-07-17 18:21:02.913435
23,Instagram,ADC,التبيض دايم ولا مؤقت وكم سعره,um_ibrahiim3030,2023-07-17 16:21:02.913435


In [50]:
filtered_df.drop_duplicates(inplace=True)

C:\Users\omar.mounir\AppData\Local\Temp\ipykernel_8120\1301694519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop_duplicates(inplace=True)


In [51]:
filtered_df

,Platform,Business Unit,Comment / Mention,Comment Author,Current date
1,FaceBook,APC,الموقع,مالك ومروان,2023-07-17 09:21:02.913435
4,FaceBook,ADC,السلام عليكم كم الحشو,ابو مصطفي ابو مصطفي,2023-07-17 18:21:02.913435
5,FaceBook,ADC,كم سعر السن الواحد,أ.احمد الوا حدي,2023-07-17 18:21:02.913435
6,FaceBook,ALW,عبارة عن ايه الجلسة من فضلك,Ghada Ibrahim,2023-07-17 18:21:02.913435
18,Instagram,ADC,ما العروض المتاحة علي تنظيف الاسنان,ysraaltayeb,2023-07-17 18:21:02.913435
19,Instagram,ADC,Okay,iinada02,2023-07-17 18:21:02.913435
20,Instagram,ADC,ما العروض المتاحة علي ال so check ؟,omalaa___13,2023-07-17 18:21:02.913435
21,Instagram,ADC,كيف احجز لاني من المدية رايحة جدة الاربعاء,anahappy6677,2023-07-17 18:21:02.913435
22,Instagram,ADC,كيف اخسر وزني؟؟,rmharasani,2023-07-17 18:21:02.913435
23,Instagram,ADC,التبيض دايم ولا مؤقت وكم سعره,um_ibrahiim3030,2023-07-17 16:21:02.913435


In [22]:
for week in range(5):
    comments = []; days = []; names = []; indices = [];dates_time = []
    for i in range(num_links):
        link = all_links[i]
        driver.get(link)
        time.sleep(20)

        htmlstring = driver.page_source
        soup = BeautifulSoup(htmlstring, "html.parser")

        long_scroll(driver=driver)

        #mydivs = soup.findAll("ap-inbox-item-list-card",class_="item-entry-container item-selected ng-star-inserted")
        mydivs = soup.find('div',class_='item-list-cards-container')

        comments.append([]); days.append([]); names.append([]); dates_time.append([])
        indices.append(all_pages[i])

        if len(mydivs) > 0:
                for div in mydivs:
                    try:
                        name = div.findNext("span",attrs={"class":"user-name"}).text.strip()
                        day = div.findNext("span",attrs={"class":"creation-time-ago ng-star-inserted"}).text.strip()
                        comment = div.findNext("span",attrs={"class":"item-card-message ng-star-inserted"}).text.strip()
                        current_date = datetime.now().date()

                        if ( len(comments[i]) == 0):
                            names[i].append(name);  days[i].append(day); comments[i].append(comment)
                            dates_time[i].append(current_date)
                        elif ( comments[i][-1] != comment):
                            names[i].append(name);  days[i].append(day); comments[i].append(comment)
                            dates_time[i].append(current_date)
                    except:
                        j = i
        else:
            comments[i] = ['']; days[i] = ['']; names[i] = ['']; dates_time[i] = ['']

    indices_column = []; names_column = []
    comments_column = []; days_column = []; dates_column = []

    for i in range(num_links):
        if len(comments[i]) > 0:
            for j in range(len(comments[i])):
                indices_column.append(indices[i])
                comments_column.append(comments[i][j]) ; dates_column.append(dates_time[i][j])
                names_column.append(names[i][j]);  days_column.append(days[i][j])

    platform, Business_Units = [], []

    for i in range(len(indices_column)):
        social_media, bu = indices_column[i].split(' ')[0], indices_column[i].split(' ')[1]
        platform.append(social_media); Business_Units.append(bu)

    df = pd.DataFrame()

    df['Platform'] = platform
    df['Business Unit'] = Business_Units
    df['message'] = comments_column
    df['Time ago'] = days_column
    df['Comment Author'] = names_column
    df['Current date'] = dates_column

    current_datetime = datetime.now()
    modified_datetime = current_datetime - timedelta(minutes=5)

    time_ago = df['Time ago'].tolist()
    dates_column = []

    for i in range(len(time_ago)):
        dates_column.append(gm_date(time_ago[i], modified_datetime))

    df['Current date'] = dates_column
    df.drop(columns=['Time ago'], inplace=True)

    datetime_str = current_datetime - timedelta(days=1)
    certain_date = pd.to_datetime(datetime_str)

    df = df[df['Current date'] >= certain_date]
    df.drop_duplicates(inplace=True)

    directory_path = 'Extracted SocialMediaComments/' + str(current_date)
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    directory_path += '/Data_Shown.csv'
    df.to_csv(directory_path, encoding='utf-8-sig', index=False)
    time.sleep(day_time_sec)

KeyboardInterrupt: 

In [52]:
filtered_df.to_csv('Extracted SocialMediaComments/All data 17July2023/v3.csv',
                   index=False,encoding='utf-8-sig')

In [76]:
indices_column = []; names_column = []
comments_column = []; days_column = []
for i in range(num_links):
    if len(comments[i]) > 0:
        for j in range(len(comments[i])):
            indices_column.append(indices[i])
            comments_column.append(comments[i][j])
            names_column.append(names[i][j]);  days_column.append(days[i][j])

In [77]:
len(indices_column)

77

In [78]:
platform, Business_Units = [], []
for i in range(len(indices_column)):
    social_media, bu = indices_column[i].split(' ')[0], indices_column[i].split(' ')[1]

    platform.append(social_media);
    Business_Units.append(bu)
df = pd.DataFrame()
df['Platform'] = platform
df['Business Unit'] = Business_Units
df['Comment / Mention'] = comments_column
df['Time ago'] = days_column
df['Comment Author'] = names_column

In [79]:
df

,Platform,Business Unit,Comment / Mention,Time ago,Comment Author
0,FaceBook,HJH,لي ما سويته لنفسك يا دكتور,19 hours ago,احمد الصعيدي
1,FaceBook,HJH,ماشاء الله تبارك الدكتور اصلع ههههه,2 days ago,عبدالله العمري
2,FaceBook,HJH,حرام عليكم دي فاشله جدا وعذاب في المعده لاتضحك...,3 days ago,Suzan Mostafa
3,FaceBook,HJH,طب اعمل لنفسك بدل الصلعة اللي عندك دي دا لو ال...,5 days ago,رقت عيناي شوقآ
4,FaceBook,HJH,خراااااااطين لا تصدقوهم\nعلاج غير معتمد من أي...,5 days ago,علي الطيب
...,...,...,...,...,...
72,Youtube,ADC,بس صوت الدكتور منخفض,2 months ago,Qwqw Qwqw
73,Youtube,ADC,❤❤❤❤😊,a month ago,Rhrc Dkr
74,Youtube,ADC,تبع مرحبا مليون مبروك يا ابو محمد وعلى آل محمد...,a month ago,Rhrc Dkr
75,Youtube,ADC,الغبي ضيع أسنانه لانه اختار تركيبة ولم يختار ز...,2 months ago,ddd ggg


In [80]:
len(df)

77

In [81]:
len(df[df['Comment / Mention'] != ''])

77

In [95]:
needed_date = dates_time[0][0]

In [96]:
dates_time = [needed_date]*77

df['Current date'] = dates_time

In [98]:
df.to_csv('Extracted SocialMediaComments/Hidden_data_17July/v2.csv', encoding='utf-8-sig', index=False)

## Merge two dataframes

In [43]:
import glob

path_files = 'Extracted SocialMediaComments/2023-08-02/*.xlsx'

path_files = glob.glob(path_files)
path_files

['Extracted SocialMediaComments/2023-08-02\\Data_Shown_24.xlsx',
 'Extracted SocialMediaComments/2023-08-02\\Data_Shown_25.xlsx']

In [45]:
dfs = []

for path1 in path_files:
    dfs.append(pd.read_excel(path1))

for i in range(len(path_files)):
    try:
        dfs[i].rename(columns={'Comment / Mention': 'message'}, inplace=True)
    except:
        print(i)

In [46]:
df2 = dfs[1]
df1 = dfs[0]

In [47]:
df2

,Platform,Business Unit,message,Comment Author,Current date
0,FaceBook,ALW,كم سعر الإبر. وهل لها تأثير عل أمراض القلب,ناصر علي ابن عدن,2023-08-02 09:18:02.738
1,FaceBook,APC,الووو,حسن الحميدي,2023-08-02 08:18:02.738
2,FaceBook,APC,تم,فضل حمزه,2023-08-02 02:18:02.738
3,FaceBook,APC,كم السعر,Adnan King,2023-08-02 02:18:02.738
4,FaceBook,APC,كيف الأمر لمريض السكري \nوهل جلسة واحدة \nوكم ...,Hamada Alsanausi Abu Yahya,2023-08-02 14:02:52.624
5,FaceBook,Andalusia_Egypt,بيحصل كدا في رجليه لو مشيت شويه أو قفت ولابسه ...,Bebo Azab,2023-08-02 13:18:02.738
6,FaceBook,Andalusia_Visiting,تفاصيل اكثر,ابو وليد,2023-08-02 12:18:02.738
7,FaceBook,Andalusia_Visiting,السعر كام,Reda Saber,2023-08-02 12:18:02.738
8,FaceBook,Andalusia_Visiting,متعاقدين مع مدرايت,Facebook user,2023-08-02 12:18:02.738
9,FaceBook,Andalusia_Visiting,تفاصيل,Cap Abdallah,2023-08-02 11:18:02.738


In [48]:
df2

,Platform,Business Unit,message,Comment Author,Current date
0,FaceBook,ALW,كم سعر الإبر. وهل لها تأثير عل أمراض القلب,ناصر علي ابن عدن,2023-08-02 09:18:02.738
1,FaceBook,APC,الووو,حسن الحميدي,2023-08-02 08:18:02.738
2,FaceBook,APC,تم,فضل حمزه,2023-08-02 02:18:02.738
3,FaceBook,APC,كم السعر,Adnan King,2023-08-02 02:18:02.738
4,FaceBook,APC,كيف الأمر لمريض السكري \nوهل جلسة واحدة \nوكم ...,Hamada Alsanausi Abu Yahya,2023-08-02 14:02:52.624
5,FaceBook,Andalusia_Egypt,بيحصل كدا في رجليه لو مشيت شويه أو قفت ولابسه ...,Bebo Azab,2023-08-02 13:18:02.738
6,FaceBook,Andalusia_Visiting,تفاصيل اكثر,ابو وليد,2023-08-02 12:18:02.738
7,FaceBook,Andalusia_Visiting,السعر كام,Reda Saber,2023-08-02 12:18:02.738
8,FaceBook,Andalusia_Visiting,متعاقدين مع مدرايت,Facebook user,2023-08-02 12:18:02.738
9,FaceBook,Andalusia_Visiting,تفاصيل,Cap Abdallah,2023-08-02 11:18:02.738


In [104]:
#df1.drop(columns=['Current date'],inplace=True)

In [49]:
merged_df = pd.merge(df1, df2, on=['Platform', 'Business Unit', 'message'],how='outer')

In [50]:
merged_df

,Platform,Business Unit,message,Comment Author_x,Current date_x,Comment Author_y,Current date_y
0,FaceBook,HJH,انت قلتم مجاني.. هل فعلا الكشف مجاني؟,لندا موسى,2023-08-02 13:38:40.070,لندا موسى,2023-08-02 14:02:52.624
1,FaceBook,HJH,اناس داني بالمنطقه الجنوبية عندالمشي الركبه ال...,حسن سراج الدين الطيب,2023-08-02 13:38:40.070,حسن سراج الدين الطيب,2023-08-02 14:02:52.624
2,FaceBook,HJH,How much to remove the tooth,Arshad Chohan,2023-08-01 21:38:40.070,Arshad Chohan,2023-08-01 21:18:02.738
3,FaceBook,APC,كيف الأمر لمريض السكري \nوهل جلسة واحدة \nوكم ...,Hamada Alsanausi Abu Yahya,2023-08-02 13:38:40.070,Hamada Alsanausi Abu Yahya,2023-08-02 14:02:52.624
4,FaceBook,APC,الووو,حسن الحميدي,2023-08-02 08:38:40.070,حسن الحميدي,2023-08-02 08:18:02.738
5,FaceBook,APC,تم,فضل حمزه,2023-08-02 01:38:40.070,فضل حمزه,2023-08-02 02:18:02.738
6,FaceBook,APC,كم السعر,Adnan King,2023-08-02 01:38:40.070,Adnan King,2023-08-02 02:18:02.738
7,FaceBook,KSA_HomeCare,ممكن تفاصيل عن الاسعار,ماما حبيتي,2023-07-29 14:38:40.070,ماما حبيتي,2023-07-29 13:18:02.738
8,FaceBook,KSA_HomeCare,بلاش توهمو الناس الحمد لله الناس بخير,عبدالله بشير,2023-07-28 14:38:40.070,عبدالله بشير,2023-07-28 13:18:02.738
9,FaceBook,KSA_HomeCare,كم سعره,Sadam Alsharabi,2023-07-27 14:38:40.070,Sadam Alsharabi,2023-07-27 13:18:02.738


In [51]:
len(merged_df)

51

In [52]:
merged_df.to_csv('Extracted SocialMediaComments/2023-08-02/v3.csv', encoding='utf-8-sig', index=False)

In [53]:
merged_df = pd.read_excel('Extracted SocialMediaComments/data to 8-02 June 2023/Agora_comments.xlsx')

merged_df

,Platform,Business Unit,message,Comment Author,Current date
0,FaceBook,HJH,انت قلتم مجاني.. هل فعلا الكشف مجاني؟,لندا موسى,2023-08-02 13:38:40.000
1,FaceBook,HJH,اناس داني بالمنطقه الجنوبية عندالمشي الركبه ال...,حسن سراج الدين الطيب,2023-08-02 13:38:40.070
2,FaceBook,HJH,How much to remove the tooth,Arshad Chohan,2023-08-01 21:38:40.070
3,FaceBook,APC,كيف الأمر لمريض السكري \nوهل جلسة واحدة \nوكم ...,Hamada Alsanausi Abu Yahya,2023-08-02 13:38:40.070
4,FaceBook,APC,الووو,حسن الحميدي,2023-08-02 08:38:40.070
5,FaceBook,APC,تم,فضل حمزه,2023-08-02 01:38:40.070
6,FaceBook,APC,كم السعر,Adnan King,2023-08-02 01:38:40.070
7,FaceBook,KSA_HomeCare,ممكن تفاصيل عن الاسعار,ماما حبيتي,2023-07-29 14:38:40.070
8,FaceBook,KSA_HomeCare,بلاش توهمو الناس الحمد لله الناس بخير,عبدالله بشير,2023-07-28 14:38:40.070
9,FaceBook,KSA_HomeCare,كم سعره,Sadam Alsharabi,2023-07-27 14:38:40.070


In [54]:
merged_df.sort_values(by=['Platform','Business Unit'], inplace=True)
merged_df

,Platform,Business Unit,message,Comment Author,Current date
30,FaceBook,ALW,كم سعر الإبر. وهل لها تأثير عل أمراض القلب,ناصر علي ابن عدن,2023-08-02 09:18:02.738
3,FaceBook,APC,كيف الأمر لمريض السكري \nوهل جلسة واحدة \nوكم ...,Hamada Alsanausi Abu Yahya,2023-08-02 13:38:40.070
4,FaceBook,APC,الووو,حسن الحميدي,2023-08-02 08:38:40.070
5,FaceBook,APC,تم,فضل حمزه,2023-08-02 01:38:40.070
6,FaceBook,APC,كم السعر,Adnan King,2023-08-02 01:38:40.070
31,FaceBook,Andalusia_Egypt,بيحصل كدا في رجليه لو مشيت شويه أو قفت ولابسه ...,Bebo Azab,2023-08-02 13:18:02.738
13,FaceBook,Andalusia_Visiting,تفاصيل اكثر,ابو وليد,2023-08-02 12:38:40.070
14,FaceBook,Andalusia_Visiting,السعر كام,Reda Saber,2023-08-02 11:38:40.070
15,FaceBook,Andalusia_Visiting,متعاقدين مع مدرايت,Facebook user,2023-08-02 11:38:40.070
16,FaceBook,Andalusia_Visiting,تفاصيل,Cap Abdallah,2023-08-02 11:38:40.070


In [55]:
merged_df.to_csv('Extracted SocialMediaComments/data to 8-02 June 2023/Agora_comments.csv', encoding='utf-8-sig', index=False)

In [189]:
merged_df_old = pd.read_csv('Extracted SocialMediaComments/16-7-2023 - 10-15/merged_16-7-2023.csv')
merged_df_old

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-16,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-16,11 hours ago,ياسين احمد
2,FaceBook,HJH,كم سعر الكشف والتحليل,2023-07-16,11 hours ago,أبو على
3,FaceBook,HJH,السعر,2023-07-16,11 hours ago,Mahmoud Emam
4,FaceBook,HJH,Andalusia Hospital - مستشفى أندلسية,2023-07-16,11 hours ago,أحمد نايف سعيد اليماني
...,...,...,...,...,...,...
73,Instagram,ALW,تفاصيل,2023-07-16,a month ago,rewanragab19
74,Instagram,ALW,تفاصيل,2023-07-16,2 months ago,nancyaslam86
75,Instagram,ALW,السلام عليكم هو فيه دورات طوارئ واسعافات اولية...,2023-07-16,2 months ago,doc.h_t
76,Instagram,AHC,وين العنوان,2023-07-16,14 hours ago,Mohammed Faheem


In [198]:
df_out = pd.merge(merged_df, merged_df_old, on=['Platform', 'Business Unit', 'Comment / Mention','Current date','Comment Author','Time ago'],how='inner')

df_out

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-16,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-16,11 hours ago,ياسين احمد
2,FaceBook,APC,ماهي أنواع الفحوصات، والسعر,2023-07-16,10 hours ago,ماجد ماجد
3,FaceBook,APC,هل يحتاج موعد أو حضور على طول,2023-07-16,11 hours ago,مصطفى القطبه
4,FaceBook,APC,متوفر عندكم تصحيح نظر,2023-07-16,12 hours ago,Mohamed Ahmed Ibra
...,...,...,...,...,...,...
95,Instagram,ALW,تفاصيل,2023-07-16,2 months ago,nancyaslam86
96,Instagram,ALW,طريقة الحجز,2023-07-16,2 months ago,yousef_gamal64
97,Instagram,ALW,السلام عليكم هو فيه دورات طوارئ واسعافات اولية...,2023-07-16,2 months ago,doc.h_t
98,Instagram,AHC,وين العنوان,2023-07-16,14 hours ago,Mohammed Faheem


In [195]:
#df_out.drop_duplicates(inplace=True)

In [199]:
df_out.to_csv('Extracted SocialMediaComments/16-7-2023_extracted_merged.csv', encoding='utf-8-sig', index=False)

# First Four days Scraping Collection - AD Hoc

In [202]:
df_13_7 = pd.read_csv('Extracted SocialMediaComments/13-7-2023/Merged/v1.csv')
df_13_7.dropna(subset=['Comment Author'], inplace=True)
df_13_7

,Platform,Business Unit,Comment / Mention,Time ago,Comment Author
1,FaceBook,HJH,السلام عليكم ورحمة الله وبركاتة\nعندكم فرع في ...,5 hours ago,سنان مجرشي
6,FaceBook,APC,بكم,4 hours ago,ام ادواء
7,FaceBook,APC,بكم الحجز,12 hours ago,عبدالله العمودي
8,FaceBook,APC,كم خلع الاسنان,14 hours ago,Shams Mohammedi
13,FaceBook,ADC,Amein,7 hours ago,Sada Gul
14,FaceBook,ADC,Good pm doc,21 hours ago,Zel Torres
15,FaceBook,AHC,تفاصيل,21 hours ago,Ahmed Hosny Zidan
16,FaceBook,AHC,تفاصيل,a day ago,مصطفي خالد
17,FaceBook,AHC,تفاصيل,a day ago,Aml Shreef
18,FaceBook,AHC,ممكن تفاصيل,a day ago,Om Mohammed Babers


In [205]:
df_16_7_v1 = pd.read_csv('Extracted SocialMediaComments/16-7-2023 - 10-15/merged_16-7-2023.csv')
df_16_7_v2 = pd.read_csv('Extracted SocialMediaComments/16-7-2023 - 10-30/v3_merged.csv')
df_16_7_v1.drop_duplicates(inplace=True)
df_16_7_v2.drop_duplicates(inplace=True)

In [206]:
df_16_7_v1

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-16,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-16,11 hours ago,ياسين احمد
2,FaceBook,HJH,كم سعر الكشف والتحليل,2023-07-16,11 hours ago,أبو على
3,FaceBook,HJH,السعر,2023-07-16,11 hours ago,Mahmoud Emam
4,FaceBook,HJH,Andalusia Hospital - مستشفى أندلسية,2023-07-16,11 hours ago,أحمد نايف سعيد اليماني
...,...,...,...,...,...,...
71,Instagram,ALW,تفاصيل,2023-07-16,2 months ago,nancyaslam86
72,Instagram,ALW,طريقة الحجز,2023-07-16,2 months ago,yousef_gamal64
75,Instagram,ALW,السلام عليكم هو فيه دورات طوارئ واسعافات اولية...,2023-07-16,2 months ago,doc.h_t
76,Instagram,AHC,وين العنوان,2023-07-16,14 hours ago,Mohammed Faheem


In [207]:
df_16_7_v2

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-16,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-16,11 hours ago,ياسين احمد
2,FaceBook,HJH,كم سعر الكشف والتحليل,2023-07-16,12 hours ago,أبو على
3,FaceBook,HJH,السعر,2023-07-16,12 hours ago,Mahmoud Emam
4,FaceBook,HJH,Andalusia Hospital - مستشفى أندلسية,2023-07-16,12 hours ago,أحمد نايف سعيد اليماني
...,...,...,...,...,...,...
74,Instagram,ALW,تفاصيل,2023-07-16,2 months ago,nancyaslam86
77,Instagram,ALW,طريقة الحجز,2023-07-16,2 months ago,yousef_gamal64
78,Instagram,ALW,السلام عليكم هو فيه دورات طوارئ واسعافات اولية...,2023-07-16,2 months ago,doc.h_t
79,Instagram,AHC,وين العنوان,2023-07-16,14 hours ago,Mohammed Faheem


In [209]:
df_16_7 = pd.merge(df_16_7_v1, df_16_7_v2, on=['Platform', 'Business Unit', 'Comment / Mention','Current date','Comment Author','Time ago'],how='outer')

df_16_7

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-16,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-16,11 hours ago,ياسين احمد
2,FaceBook,HJH,كم سعر الكشف والتحليل,2023-07-16,11 hours ago,أبو على
3,FaceBook,HJH,السعر,2023-07-16,11 hours ago,Mahmoud Emam
4,FaceBook,HJH,Andalusia Hospital - مستشفى أندلسية,2023-07-16,11 hours ago,أحمد نايف سعيد اليماني
...,...,...,...,...,...,...
81,Instagram,APC,بكم,2023-07-16,22 minutes ago,hfmt2022
82,Instagram,APC,ممكن لو سمحتي التفاصيل وهل تقبلون الطفل وهو عم...,2023-07-16,12 hours ago,aseell.al7rbe
83,Instagram,AKW,كم السعر,2023-07-16,18 hours ago,sweets_morocco_argane_jeddah
84,Instagram,ADC,كم سعر شفط الفخدين بالفيزر,2023-07-16,2 days ago,rahema_2244


In [217]:
previous_date = datetime.now().date() - timedelta(days=3)

previous_date

datetime.date(2023, 7, 13)

In [218]:
df_13_7['Current date'] =previous_date
df_13_7

,Platform,Business Unit,Comment / Mention,Time ago,Comment Author,Current date
1,FaceBook,HJH,السلام عليكم ورحمة الله وبركاتة\nعندكم فرع في ...,5 hours ago,سنان مجرشي,2023-07-13
6,FaceBook,APC,بكم,4 hours ago,ام ادواء,2023-07-13
7,FaceBook,APC,بكم الحجز,12 hours ago,عبدالله العمودي,2023-07-13
8,FaceBook,APC,كم خلع الاسنان,14 hours ago,Shams Mohammedi,2023-07-13
13,FaceBook,ADC,Amein,7 hours ago,Sada Gul,2023-07-13
14,FaceBook,ADC,Good pm doc,21 hours ago,Zel Torres,2023-07-13
15,FaceBook,AHC,تفاصيل,21 hours ago,Ahmed Hosny Zidan,2023-07-13
16,FaceBook,AHC,تفاصيل,a day ago,مصطفي خالد,2023-07-13
17,FaceBook,AHC,تفاصيل,a day ago,Aml Shreef,2023-07-13
18,FaceBook,AHC,ممكن تفاصيل,a day ago,Om Mohammed Babers,2023-07-13


In [224]:
df_16_7 = df_16_7[df_16_7['Time ago'] != '3 days ago']
df_16_7 = df_16_7[df_16_7['Time ago'] != '4 days ago']
df_16_7 = df_16_7[df_16_7['Time ago'] != '5 days ago']
df_16_7 = df_16_7[df_16_7['Time ago'] != '8 days ago']
df_16_7 = df_16_7[df_16_7['Time ago'] != '15 days ago']
df_16_7 = df_16_7[df_16_7['Time ago'] != '16 days ago']
df_16_7 = df_16_7[df_16_7['Time ago'] != 'a month ago']
df_16_7 = df_16_7[df_16_7['Time ago'] != '2 months ago']
df_16_7

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-16,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-16,11 hours ago,ياسين احمد
2,FaceBook,HJH,كم سعر الكشف والتحليل,2023-07-16,11 hours ago,أبو على
3,FaceBook,HJH,السعر,2023-07-16,11 hours ago,Mahmoud Emam
4,FaceBook,HJH,Andalusia Hospital - مستشفى أندلسية,2023-07-16,11 hours ago,أحمد نايف سعيد اليماني
...,...,...,...,...,...,...
81,Instagram,APC,بكم,2023-07-16,22 minutes ago,hfmt2022
82,Instagram,APC,ممكن لو سمحتي التفاصيل وهل تقبلون الطفل وهو عم...,2023-07-16,12 hours ago,aseell.al7rbe
83,Instagram,AKW,كم السعر,2023-07-16,18 hours ago,sweets_morocco_argane_jeddah
84,Instagram,ADC,كم سعر شفط الفخدين بالفيزر,2023-07-16,2 days ago,rahema_2244


In [223]:
df_16_7['Time ago'].tolist()

['11 hours ago',
 '11 hours ago',
 '11 hours ago',
 '11 hours ago',
 '11 hours ago',
 '11 hours ago',
 '6 hours ago',
 '6 hours ago',
 '10 hours ago',
 '11 hours ago',
 '12 hours ago',
 '2 days ago',
 '10 hours ago',
 '12 hours ago',
 '12 hours ago',
 '12 hours ago',
 '13 hours ago',
 '14 hours ago',
 '14 hours ago',
 '13 hours ago',
 '15 hours ago',
 '15 hours ago',
 '16 hours ago',
 '19 hours ago',
 'a day ago',
 '2 hours ago',
 '4 hours ago',
 '6 hours ago',
 '7 hours ago',
 '8 hours ago',
 '9 hours ago',
 '8 hours ago',
 '9 hours ago',
 '9 hours ago',
 '10 hours ago',
 '10 hours ago',
 '11 hours ago',
 '3 hours ago',
 '4 hours ago',
 '11 hours ago',
 '11 hours ago',
 '12 hours ago',
 '12 hours ago',
 '6 hours ago',
 '10 hours ago',
 '17 hours ago',
 '18 hours ago',
 '20 hours ago',
 'a day ago',
 '2 days ago',
 '2 days ago',
 '2 days ago',
 '2 days ago',
 '2 days ago',
 '14 hours ago',
 '8 days ago',
 '12 hours ago',
 '12 hours ago',
 '12 hours ago',
 '12 hours ago',
 '3 days ago',

In [226]:
df_since_13_7 = pd.merge(df_16_7, df_13_7, on=['Platform', 'Business Unit', 'Comment / Mention','Current date','Comment Author','Time ago'],how='outer')
df_since_13_7

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-16,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-16,11 hours ago,ياسين احمد
2,FaceBook,HJH,كم سعر الكشف والتحليل,2023-07-16,11 hours ago,أبو على
3,FaceBook,HJH,السعر,2023-07-16,11 hours ago,Mahmoud Emam
4,FaceBook,HJH,Andalusia Hospital - مستشفى أندلسية,2023-07-16,11 hours ago,أحمد نايف سعيد اليماني
...,...,...,...,...,...,...
109,Instagram,ALW,تفاصيل,2023-07-13,2 months ago,nancyaslam86
110,Instagram,ALW,تفاصيل,2023-07-13,2 months ago,nancyaslam86
111,Instagram,ALW,طريقة الحجز,2023-07-13,2 months ago,yousef_gamal64
112,Instagram,ALW,السلام عليكم هو فيه دورات طوارئ واسعافات اولية...,2023-07-13,2 months ago,doc.h_t


In [229]:
df_since_13_7 = df_since_13_7.sort_values(by=['Platform', 'Business Unit'], ascending=[True, False])
df_since_13_7

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-16,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-16,11 hours ago,ياسين احمد
2,FaceBook,HJH,كم سعر الكشف والتحليل,2023-07-16,11 hours ago,أبو على
3,FaceBook,HJH,السعر,2023-07-16,11 hours ago,Mahmoud Emam
4,FaceBook,HJH,Andalusia Hospital - مستشفى أندلسية,2023-07-16,11 hours ago,أحمد نايف سعيد اليماني
...,...,...,...,...,...,...
96,Twitter,AHC,كم سعرها,2023-07-13,2 hours ago,ghadah.3333
97,Twitter,AHC,كم السعر,2023-07-13,3 hours ago,bin_jamaan
98,Twitter,AHC,كم سعرها ؟؟,2023-07-13,5 hours ago,remarema85
99,Twitter,AHC,بكم السعر,2023-07-13,5 hours ago,sugar_gir222


In [234]:
df_since_13_7.to_csv('Extracted SocialMediaComments/since_13_7.csv',index=False,encoding='utf-8-sig')

In [233]:
df_since_13_7.drop_duplicates(inplace=True)

In [241]:
import re

def gm_date(rev_date,needed_date):
    days_vol = {"day": 1, "week": 7, "month": 30, "year": 365}
    text = rev_date.split()
    text[0] = re.sub(r'an?', '1', text[0])
    text[1] = text[1].rstrip('s')
    num_hours = int(text[0]) * int(text[1] == "hour")

    try:
        duration = days_vol[text[1]]
    except KeyError:
        duration = 0
    num_days = int(text[0])*duration

    return get_day(needed_date,num_days, num_hours, day_only=False)

def get_day(needed_date, past_days=0, past_hours=0, day_only=True,):
    ret = needed_date - timedelta(days=past_days, hours=past_hours)

    return ret.date() if day_only else ret

In [254]:
datetime.now().date()

datetime.date(2023, 7, 16)

In [256]:
datetime.now().date().strftime('%Y-%m-%d %H:%M')

'2023-07-16 00:00'

In [335]:
current_datetime = datetime.now()

modified_datetime = current_datetime - timedelta(days=2,hours=20,minutes=44)

# Format the modified datetime
#previous_date = modified_datetime.strftime('%Y-%m-%d %H:%M')
modified_datetime

datetime.datetime(2023, 7, 13, 17, 36, 49, 385136)

In [336]:
current_datetime = datetime.now()

current_datetime = current_datetime - timedelta(hours=3,minutes=44)

# Format the modified datetime
#previous_date = modified_datetime.strftime('%Y-%m-%d %H:%M')
current_datetime

datetime.datetime(2023, 7, 16, 10, 37, 2, 418854)

In [311]:
str(df_since_13_7['Current date'].tolist()[0]) == '2023-07-16'

True

In [325]:
current_datetime

'2023-07-16 10:33'

In [324]:
previous_datetime =

'2023-07-13 17:36'

In [339]:
current_date_column[i]

'2023-07-16'

In [344]:
time_ago = df_since_13_7['Time ago'].tolist()
current_date_column = df_since_13_7['Current date'].tolist()

In [345]:

dates_column = []

for i in range(len(current_date_column)):

    if str(current_date_column[i]) == '2023-07-16':
        dates_column.append(gm_date( time_ago[i] ,current_datetime))
    elif str(current_date_column[i]) == '2023-07-13':
        dates_column.append(gm_date( time_ago[i] ,modified_datetime))

In [346]:
dates_column

[datetime.datetime(2023, 7, 15, 23, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 23, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 23, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 23, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 23, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 23, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 22, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 22, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 22, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 22, 37, 2, 418854),
 datetime.datetime(2023, 7, 13, 12, 36, 49, 385136),
 datetime.datetime(2023, 7, 16, 4, 37, 2, 418854),
 datetime.datetime(2023, 7, 16, 4, 37, 2, 418854),
 datetime.datetime(2023, 7, 16, 0, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 23, 37, 2, 418854),
 datetime.datetime(2023, 7, 15, 22, 37, 2, 418854),
 datetime.datetime(2023, 7, 14, 10, 37, 2, 418854),
 datetime.datetime(2023, 7, 13, 10, 37, 2, 418854),
 datetime.datetime(2023, 7, 13, 13, 36, 49, 385136),
 datetime.dat

In [347]:
df_since_13_7

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-16,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-16,11 hours ago,ياسين احمد
2,FaceBook,HJH,كم سعر الكشف والتحليل,2023-07-16,11 hours ago,أبو على
3,FaceBook,HJH,السعر,2023-07-16,11 hours ago,Mahmoud Emam
4,FaceBook,HJH,Andalusia Hospital - مستشفى أندلسية,2023-07-16,11 hours ago,أحمد نايف سعيد اليماني
...,...,...,...,...,...,...
95,Twitter,AHC,كم سعرها,2023-07-13,2 hours ago,ghadah.3333
97,Twitter,AHC,كم السعر,2023-07-13,3 hours ago,bin_jamaan
98,Twitter,AHC,كم سعرها ؟؟,2023-07-13,5 hours ago,remarema85
99,Twitter,AHC,بكم السعر,2023-07-13,5 hours ago,sugar_gir222


In [348]:
df_since_13_7['Current date'] = dates_column
df_since_13_7

,Platform,Business Unit,Comment / Mention,Current date,Time ago,Comment Author
0,FaceBook,HJH,كم سعر الكشف,2023-07-15 23:37:02.418854,11 hours ago,عادل النجار
1,FaceBook,HJH,وين مكانه,2023-07-15 23:37:02.418854,11 hours ago,ياسين احمد
2,FaceBook,HJH,كم سعر الكشف والتحليل,2023-07-15 23:37:02.418854,11 hours ago,أبو على
3,FaceBook,HJH,السعر,2023-07-15 23:37:02.418854,11 hours ago,Mahmoud Emam
4,FaceBook,HJH,Andalusia Hospital - مستشفى أندلسية,2023-07-15 23:37:02.418854,11 hours ago,أحمد نايف سعيد اليماني
...,...,...,...,...,...,...
95,Twitter,AHC,كم سعرها,2023-07-13 15:36:49.385136,2 hours ago,ghadah.3333
97,Twitter,AHC,كم السعر,2023-07-13 14:36:49.385136,3 hours ago,bin_jamaan
98,Twitter,AHC,كم سعرها ؟؟,2023-07-13 12:36:49.385136,5 hours ago,remarema85
99,Twitter,AHC,بكم السعر,2023-07-13 12:36:49.385136,5 hours ago,sugar_gir222


In [350]:
df_since_13_7.drop(columns=['Time ago'],inplace=True)

In [354]:
df_since_13_7_2 = df_since_13_7[['Platform','Business Unit','Comment Author','Comment / Mention','Current date']]
df_since_13_7_2

,Platform,Business Unit,Comment Author,Comment / Mention,Current date
0,FaceBook,HJH,عادل النجار,كم سعر الكشف,2023-07-15 23:37:02.418854
1,FaceBook,HJH,ياسين احمد,وين مكانه,2023-07-15 23:37:02.418854
2,FaceBook,HJH,أبو على,كم سعر الكشف والتحليل,2023-07-15 23:37:02.418854
3,FaceBook,HJH,Mahmoud Emam,السعر,2023-07-15 23:37:02.418854
4,FaceBook,HJH,أحمد نايف سعيد اليماني,Andalusia Hospital - مستشفى أندلسية,2023-07-15 23:37:02.418854
...,...,...,...,...,...
95,Twitter,AHC,ghadah.3333,كم سعرها,2023-07-13 15:36:49.385136
97,Twitter,AHC,bin_jamaan,كم السعر,2023-07-13 14:36:49.385136
98,Twitter,AHC,remarema85,كم سعرها ؟؟,2023-07-13 12:36:49.385136
99,Twitter,AHC,sugar_gir222,بكم السعر,2023-07-13 12:36:49.385136


In [355]:
df_since_13_7_2.to_csv('Extracted SocialMediaComments/since_13_7.csv',index=False,encoding='utf-8-sig')

In [243]:
gm_date(df_since_13_7['Time ago'].values[0],needed_date= previous_date)

datetime.date(2023, 7, 13)

In [358]:
df_since_13_7_2.to_csv('Extracted SocialMediaComments/Agora-pulse/Agora_comments.csv',index=False,encoding='utf-8-sig')

In [245]:
df_since_13_7['Time ago'].values[0]

'11 hours ago'

In [ ]:
for i in range(len(comments)):
    comments[i]

In [314]:
comments = []; indices = []

for i in range(6):
    link = fb_links[i]
    driver.get(link)
    time.sleep(10)

    htmlstring = driver.page_source
    soup = BeautifulSoup(htmlstring, "html.parser")
    #wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'ng-input')))
    #mydivs = soup.findAll("span",class_="item-card-message ng-star-inserted")

    long_scroll(driver=driver)
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME,'ng-input')))
    mydivs = soup.findAll("span",class_="item-card-message ng-star-inserted")


    # mydivs = list(mydivs + mydivs2)

    comments.append([])
    indices.append(fb_pages[i])
    print(mydivs[:])
    '''
    if len(mydivs) >0:
            for div in mydivs:
                try:
                    comments[i].append(str(div).split('">')[-1].split('</')[0].strip())
                except:
                    j = i

    else:
        comments[i] = ['']
    '''

KeyboardInterrupt: 

In [199]:
comments

[['كم السعر',
  'أرغب في الحجز مباشرة',
  'سلام عليكم ورحمة الله',
  'ما العروض المتاحة علي جهاز المورفيس ؟'],
 ['حجز العرض',
  'حجز العرض',
  'السلام عليكم \nعندي مشكله في العلاقه\nبعاني من سرعة القذف \nممكن تفيدوني\nوانا  نازل مصر  بعد ثلاث  ايام',
  'Hi'],
 ['ايش اسمك انت'],
 ['ما هى العروض المتاحة حاليا',
  'ابغى اعرف كم سعره',
  'كام تكلفة تركيب فك كامل',
  'أين موقعكم',
  'أين يوجد زراعة الاسنان'],
 ['Can you sent all details pls', 'details please'],
 ['Can I learn more about your course?',
  'Details',
  'Details',
  'بستفسر عن  كورس Pediatric Dentistry Program',
  'طيب']]

In [218]:
mydivs

[<span _ngcontent-vgo-c311="" class="dot ng-star-inserted"></span>,
 <span _ngcontent-vgo-c311="" class="dot ng-star-inserted"></span>,
 <span _ngcontent-vgo-c135="" border="" class="ap-avatar ap-avatar-username small rounded ng-star-inserted"><ap-symbol _ngcontent-vgo-c135="" _nghost-vgo-c94="" class="ng-star-inserted"><div _ngcontent-vgo-c94="" class="ap-symbol micro"><div _ngcontent-vgo-c94="" class="svg"><svg viewbox="0 0 16 16" xmlns="http://www.w3.org/2000/svg"><path clip-rule="evenodd" d="M8.111 2.083a2.361 2.361 0 1 0 0 4.723 2.361 2.361 0 0 0 0-4.723ZM4.25 4.444a3.861 3.861 0 1 1 7.722 0 3.861 3.861 0 0 1-7.722 0Z" fill-rule="evenodd"></path><path clip-rule="evenodd" d="M8.111 9.806a5.028 5.028 0 0 0-5.028 5.027.75.75 0 0 1-1.5 0 6.528 6.528 0 1 1 13.056 0 .75.75 0 0 1-1.5 0A5.028 5.028 0 0 0 8.11 9.806Z" fill-rule="evenodd"></path></svg></div><!-- --></div></ap-symbol><!-- --><!-- --></span>,
 <span _ngcontent-vgo-c135="" border="cool-green" class="ap-avatar ap-avatar-usernam

In [226]:
mydivs = soup.findAll("span",class_="ng-star-inserted")
mydivs

[<div _ngcontent-vgo-c459="" class="item-list-cards-container">
 <ap-inbox-item-list-card _ngcontent-vgo-c459="" _nghost-vgo-c454="" class="item-entry-container ng-star-inserted" data-test="inbox-listItem-card" id="inboxListItemCardTrack"><div _ngcontent-vgo-c454="" class="inbox-item-list-card" mat-list-item="" selected="false" tabindex="-1">
 <div _ngcontent-vgo-c454="" class="item-card-header">
 <ap-avatar _ngcontent-vgo-c454="" _nghost-vgo-c135=""><div _ngcontent-vgo-c135="" class="ap-avatar-container medium-sm" style="width: 36px;"><span _ngcontent-vgo-c135="" border="" class="ap-avatar ap-avatar-username medium-sm outer-border rounded ng-star-inserted"> ع <!-- --><!-- --><!-- --></span><!-- --><!-- --><div _ngcontent-vgo-c135="" class="facebook medium-sm social-symbol-wrapper ng-star-inserted"><ap-symbol _ngcontent-vgo-c135="" _nghost-vgo-c94=""><div _ngcontent-vgo-c94="" class="ap-symbol" style="width: 12px; height: 12px; min-width: 12px; max-width: 12px; max-height: 12px;"><div 

In [228]:
mydivs = soup.find("div",class_="item-list-cards-container")
mydivs

<div _ngcontent-vgo-c459="" class="item-list-cards-container">
<ap-inbox-item-list-card _ngcontent-vgo-c459="" _nghost-vgo-c454="" class="item-entry-container ng-star-inserted" data-test="inbox-listItem-card" id="inboxListItemCardTrack"><div _ngcontent-vgo-c454="" class="inbox-item-list-card" mat-list-item="" selected="false" tabindex="-1">
<div _ngcontent-vgo-c454="" class="item-card-header">
<ap-avatar _ngcontent-vgo-c454="" _nghost-vgo-c135=""><div _ngcontent-vgo-c135="" class="ap-avatar-container medium-sm" style="width: 36px;"><span _ngcontent-vgo-c135="" border="" class="ap-avatar ap-avatar-username medium-sm outer-border rounded ng-star-inserted"> ع <!-- --><!-- --><!-- --></span><!-- --><!-- --><div _ngcontent-vgo-c135="" class="facebook medium-sm social-symbol-wrapper ng-star-inserted"><ap-symbol _ngcontent-vgo-c135="" _nghost-vgo-c94=""><div _ngcontent-vgo-c94="" class="ap-symbol" style="width: 12px; height: 12px; min-width: 12px; max-width: 12px; max-height: 12px;"><div _ngc

In [266]:
driver.get('https://app.agorapulse.com/inboxV2/facebook/170260?item=t_910533399799039')
time.sleep(10)

htmlstring = driver.page_source
soup = BeautifulSoup(htmlstring, "html.parser")
#wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'ng-input')))
#mydivs = soup.findAll("span",class_="item-card-message ng-star-inserted")

long_scroll(driver=driver)
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME,'ng-input')))
mydivs = soup.findAll("span",class_="ng-star-inserted")

In [267]:
comments=soup.find_all("ap-inbox-item-list-card",attrs={'id':"inboxListItemCardTrack"})

In [268]:
comments

[<ap-inbox-item-list-card _ngcontent-qqd-c459="" _nghost-qqd-c454="" class="item-entry-container item-selected ng-star-inserted" data-test="inbox-listItem-card" id="inboxListItemCardTrack"><div _ngcontent-qqd-c454="" class="inbox-item-list-card active" mat-list-item="" selected="true" tabindex="-1">
 <div _ngcontent-qqd-c454="" class="item-card-header">
 <ap-avatar _ngcontent-qqd-c454="" _nghost-qqd-c135=""><div _ngcontent-qqd-c135="" class="ap-avatar-container medium-sm" style="width: 36px;"><span _ngcontent-qqd-c135="" border="" class="ap-avatar ap-avatar-username medium-sm outer-border rounded ng-star-inserted"> د <!-- --><!-- --><!-- --></span><!-- --><!-- --><div _ngcontent-qqd-c135="" class="facebook medium-sm social-symbol-wrapper ng-star-inserted"><ap-symbol _ngcontent-qqd-c135="" _nghost-qqd-c94=""><div _ngcontent-qqd-c94="" class="ap-symbol" style="width: 12px; height: 12px; min-width: 12px; max-width: 12px; max-height: 12px;"><div _ngcontent-qqd-c94="" class="svg"><svg fill=

In [271]:
comments[0]

<ap-inbox-item-list-card _ngcontent-qqd-c459="" _nghost-qqd-c454="" class="item-entry-container item-selected ng-star-inserted" data-test="inbox-listItem-card" id="inboxListItemCardTrack"><div _ngcontent-qqd-c454="" class="inbox-item-list-card active" mat-list-item="" selected="true" tabindex="-1">
<div _ngcontent-qqd-c454="" class="item-card-header">
<ap-avatar _ngcontent-qqd-c454="" _nghost-qqd-c135=""><div _ngcontent-qqd-c135="" class="ap-avatar-container medium-sm" style="width: 36px;"><span _ngcontent-qqd-c135="" border="" class="ap-avatar ap-avatar-username medium-sm outer-border rounded ng-star-inserted"> د <!-- --><!-- --><!-- --></span><!-- --><!-- --><div _ngcontent-qqd-c135="" class="facebook medium-sm social-symbol-wrapper ng-star-inserted"><ap-symbol _ngcontent-qqd-c135="" _nghost-qqd-c94=""><div _ngcontent-qqd-c94="" class="ap-symbol" style="width: 12px; height: 12px; min-width: 12px; max-width: 12px; max-height: 12px;"><div _ngcontent-qqd-c94="" class="svg"><svg fill="no

In [307]:
def is_comment(item_list_card:BeautifulSoup):
    message_view_box="0 0 24 24"
    return item_list_card.find("div",class_="item-card-subtitle").find_next('svg').find_next().find_next_sibling()is None

In [277]:
comments[0].find('svg')

<svg fill="none" viewbox="0 0 16 16" xmlns="http://www.w3.org/2000/svg"><circle cx="8" cy="8" fill="#fff" r="8"></circle><path d="M16 7.996C16 3.584 12.414 0 8 0S0 3.584 0 7.996c0 3.994 2.93 7.299 6.753 7.897V10.3H4.71V7.988h2.043V6.224c0-2 1.19-3.116 3.011-3.116.878 0 1.78.172 1.78.172v1.952h-1c-.993 0-1.313.623-1.313 1.255v1.5h2.223l-.353 2.313H9.24v5.593C13.071 15.295 16 11.99 16 7.996Z" fill="#1877F2"></path></svg>

In [303]:
comments[2].find("div",class_="item-card-subtitle").find_next('svg').find_all_next()

595

In [294]:
comments[2]

<ap-inbox-item-list-card _ngcontent-qqd-c459="" _nghost-qqd-c454="" class="item-entry-container ng-star-inserted" data-test="inbox-listItem-card" id="inboxListItemCardTrack"><div _ngcontent-qqd-c454="" class="inbox-item-list-card" mat-list-item="" selected="false" tabindex="-1">
<div _ngcontent-qqd-c454="" class="item-card-header">
<ap-avatar _ngcontent-qqd-c454="" _nghost-qqd-c135=""><div _ngcontent-qqd-c135="" class="ap-avatar-container medium-sm" style="width: 36px;"><span _ngcontent-qqd-c135="" border="" class="ap-avatar ap-avatar-username medium-sm outer-border rounded ng-star-inserted"> B <!-- --><!-- --><!-- --></span><!-- --><!-- --><div _ngcontent-qqd-c135="" class="facebook medium-sm social-symbol-wrapper ng-star-inserted"><ap-symbol _ngcontent-qqd-c135="" _nghost-qqd-c94=""><div _ngcontent-qqd-c94="" class="ap-symbol" style="width: 12px; height: 12px; min-width: 12px; max-width: 12px; max-height: 12px;"><div _ngcontent-qqd-c94="" class="svg"><svg fill="none" viewbox="0 0 16 

In [313]:
is_comment(comments[7])

False

In [249]:
combo_box=soup.find("div",attrs={'role':'combobox','class':"ng-input"})

In [250]:
len(combo_box)

1

In [208]:
len(mydivs)

31

In [204]:
mynames = soup.findAll("span",class_="item-card-title")
mynames

[<span _ngcontent-hqk-c454="" class="item-card-title">
 <span _ngcontent-hqk-c454="" class="user-name">
                     عبدالله كستلو
                 </span>
 <!-- -->
 </span>,
 <span _ngcontent-hqk-c454="" class="item-card-title">
 <span _ngcontent-hqk-c454="" class="user-name">
                     حمادة شلبي
                 </span>
 <!-- -->
 </span>,
 <span _ngcontent-hqk-c454="" class="item-card-title">
 <span _ngcontent-hqk-c454="" class="user-name">
                     Dr-Reem Eladl
                 </span>
 <!-- -->
 </span>,
 <span _ngcontent-hqk-c454="" class="item-card-title">
 <span _ngcontent-hqk-c454="" class="user-name">
                     Dr-Reem Eladl
                 </span>
 <!-- -->
 </span>,
 <span _ngcontent-hqk-c454="" class="item-card-title">
 <span _ngcontent-hqk-c454="" class="user-name">
                     Nancy Mohamed
                 </span>
 <!-- -->
 </span>,
 <span _ngcontent-hqk-c454="" class="item-card-title">
 <span _ngcontent-hqk-c454="

In [205]:
str(mynames[0]).split('">')[-1].split('</')[0].strip()

'عبدالله كستلو'

In [209]:
len(mynames)

6

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

In [7]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome('chromedriver.exe')

AttributeError: 'str' object has no attribute '_ignore_local_proxy'